In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import pygsheets
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from tqdm import tqdm
import time
import datetime as dt
# Multi-processing
from multiprocessing import Pool, cpu_count
import warnings
warnings.filterwarnings("ignore")

credentials = Credentials.from_service_account_file('C:/Users/leemn/Documents/Loop Service Account/cb-prod-297913-623cea238b9a.json')

In [4]:
data = pd.read_excel('C:/Users/leemn/Downloads/LoopDriverDistance20221122.xlsx')

In [5]:
data.head()

,date,order_no,TipDividedBy2,actualKM,predictedKM,distanceKM,user_type,pickup_tracking_url,delivery_tracking_url,pickup_started,pickup_completed,delivery_started,delivery_completed,driver,name,order_type_id,vendor,mall,city,onTime
0,2022-11-18,3111843.0,5.0,0.0,8.90,9.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:35:33,2022-11-18 08:43:49,2022-11-18 08:43:53,2022-11-18 09:05:12,Kande Matonda,Kande,2.0,NaN,Woodbridge Square Mall,Johannesburg,yes
1,2022-11-18,3111906.0,5.0,0.0,4.53,5.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:51:47,2022-11-18 09:07:01,2022-11-18 09:07:15,2022-11-18 09:32:02,Ntobeko Mhlongo,Ntobeko,2.0,NaN,Gateway Mall,Durban,yes
2,2022-11-18,3112123.0,5.0,0.0,3.34,4.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:28:24,2022-11-18 08:45:46,2022-11-18 08:48:38,2022-11-18 09:02:19,Jackson Mzoza,Jackson,2.0,NaN,Westville Mall,Durban,yes
3,2022-11-18,3112161.0,5.0,0.0,5.06,6.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:35:55,2022-11-18 08:41:47,2022-11-18 08:42:48,2022-11-18 08:53:59,Tharcisse Muamba,Tharcisse,2.0,NaN,Woodbridge Square Mall,Johannesburg,yes
4,2022-11-18,3112208.0,5.0,0.0,5.51,6.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 13:08:44,2022-11-18 13:18:06,2022-11-18 13:18:36,2022-11-18 13:34:06,Premlal Ramraj,Premlal,2.0,NaN,Gateway Mall,Durban,yes


In [6]:
data['order_no'] = round(np.floor(pd.to_numeric(data['order_no'], errors='coerce')).astype('int'),0)

In [7]:
data['order_no'] = data['order_no'].astype('str')

In [8]:
idproductlistv = list(data.order_no.unique())
#convert the list to a tuple
p = tuple(idproductlistv)

query_trips = '''
SELECT trip_client_id as client_id, order_no, order_id, trip_id
--, delivery_distance
FROM `cb-prod-297913.cb_prod_analytics.cb_end_orders_in_trip` 
WHERE order_no IN {}

'''.format(p)

df = pd.read_gbq(query_trips, project_id='cb-prod-297913', dialect='standard', credentials=credentials,progress_bar_type='tqdm_notebook')

Downloading:   0%|          | 0/1678 [00:00<?, ?rows/s]

In [9]:
df.head()

,client_id,order_no,order_id,trip_id
0,B0l6eGBH0UB7wOhWzZ9J,3112893,qCRJBzvA7nzGALZKB0xG,72vNEudOVp7Wknv6jelg
1,B0l6eGBH0UB7wOhWzZ9J,3113986,mqdnjaHs9kujUUgf1vHW,0EzwtdcEvfhtFfdxLFkO
2,B0l6eGBH0UB7wOhWzZ9J,3115650,tRyqXak1WMxhiOStTD51,irKgwkLtsWyIPnUY9qzH
3,B0l6eGBH0UB7wOhWzZ9J,3114120,66Y9KpaP6NxfYwhsOE7H,gx2ndcnUKnHFcmDjVFUi
4,B0l6eGBH0UB7wOhWzZ9J,3113303,EqIiKHZXYDjefb2JgpBw,evU4iiEMRtxH4toadDCA


In [10]:
df_final = pd.merge(data, df, how='left', on=['order_no'])
df_final.head()

,date,order_no,TipDividedBy2,actualKM,predictedKM,distanceKM,user_type,pickup_tracking_url,delivery_tracking_url,pickup_started,...,driver,name,order_type_id,vendor,mall,city,onTime,client_id,order_id,trip_id
0,2022-11-18,3111843,5.0,0.0,8.90,9.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:35:33,...,Kande Matonda,Kande,2.0,NaN,Woodbridge Square Mall,Johannesburg,yes,B0l6eGBH0UB7wOhWzZ9J,dZ98RYwMCdQyY4nYAdkB,Brnk9aHrN1lWTVDrdVKY
1,2022-11-18,3111906,5.0,0.0,4.53,5.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:51:47,...,Ntobeko Mhlongo,Ntobeko,2.0,NaN,Gateway Mall,Durban,yes,B0l6eGBH0UB7wOhWzZ9J,aw9GJite668Q3hPNZQ73,BA7iqR3iHXhFA0Q6Nr2w
2,2022-11-18,3112123,5.0,0.0,3.34,4.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:28:24,...,Jackson Mzoza,Jackson,2.0,NaN,Westville Mall,Durban,yes,B0l6eGBH0UB7wOhWzZ9J,QJNCbxNTPHoHG87jwHdV,mtHI1QUoSaghkP9XZPui
3,2022-11-18,3112161,5.0,0.0,5.06,6.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 08:35:55,...,Tharcisse Muamba,Tharcisse,2.0,NaN,Woodbridge Square Mall,Johannesburg,yes,B0l6eGBH0UB7wOhWzZ9J,JWi2crb4S4CjF3tjBVYc,KwtFodIFeeKyeTGFlI0i
4,2022-11-18,3112208,5.0,0.0,5.51,6.0,13.0,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,https://cb-order-trackingv1-f5dovyimaq-ew.a.ru...,2022-11-18 13:08:44,...,Premlal Ramraj,Premlal,2.0,NaN,Gateway Mall,Durban,yes,B0l6eGBH0UB7wOhWzZ9J,ETzfdJ6Acpr6VPuxE18X,aSMJ0Q5DgRZU7qaCXgqL


In [11]:
df_final.to_csv('df_final_woolies_dash20221122.csv')